In [22]:
import pandas as pd
from sqlalchemy import create_engine
from password import password

In [23]:
#reading csv files into python
Artist_File_Path='Resources/artistDf.csv'
Billboard_File_Path='Resources/billboardHot100_1999-2019_sample.csv'
Grammy_File_Path='Resources/grammySongs_1999-2019.csv'

Adf=pd.read_csv(Artist_File_Path)
Bdf=pd.read_csv(Billboard_File_Path)
Gdf=pd.read_csv(Grammy_File_Path)


In [24]:
#eliminate extra columns
Adf=Adf[['Artist','Followers','Genres','NumAlbums','Gender']]
Bdf=Bdf[['Artists',	'Name',	'Weekly.rank',	'Peak.position'	,	'Date',	'Genre']]
Gdf=Gdf[['GrammyAward',	'GrammyYear',	'Genre',	'Name',	'Artist']]
Adf

,Artist,Followers,Genres,NumAlbums,Gender
0,Ed Sheeran,52698756,"pop,uk pop",8,M
1,Justin Bieber,30711450,"canadian pop,dance pop,pop,post-teen pop",10,M
2,Jonas Brothers,3069527,"boy band,dance pop,pop,post-teen pop",10,M
3,Drake,41420478,"canadian hip hop,canadian pop,hip hop,pop rap,...",11,M
4,Chris Brown,9676862,"dance pop,pop,pop rap,r&b,rap",6,M
...,...,...,...,...,...
1030,Dwight Yoakam,284597,"country,country rock,nashville sound,outlaw co...",25,M
1031,Jordan Knight,47229,NaN,4,M
1032,Lila McCann,42444,"contemporary country,country,country dawn",4,F
1033,Everlast,200550,rap rock,11,M


In [25]:
Bdf=Bdf.rename(columns={"Weekly.rank":"Weekly_rank","Peak.position":"Peak_position"})
Bdf


,Artists,Name,Weekly_rank,Peak_position,Date,Genre
0,"Lil Nas,",Old Town Road,1,1.0,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr..."
1,"Shawn Mendes, Camila Cabello",Senorita,2,NaN,"June 21, 2019",Pop
2,Billie Eilish,Bad Guy,3,2.0,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ..."
3,Khalid,Talk,4,3.0,"February 7, 2019","Synth-Pop,Pop"
4,"Ed Sheeran, Justin Bieber",I Don't Care,5,2.0,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop"
...,...,...,...,...,...,...
24995,ScHoolboy Q,Studio,75,75.0,"February 25, 2014","West Coast,Rap"
24996,Dustin Lynch,Where It's At,76,76.0,"March 31, 2014","Rock,Country"
24997,Craig Campbell,Keep Them Kisses Comin',77,72.0,NaN,Pop
24998,Lorde,Tennis Court,78,71.0,"June 7, 2013","Alternative Pop,Alternative,Downtempo,New Zeal..."


In [26]:
Gdf

,GrammyAward,GrammyYear,Genre,Name,Artist
0,Record Of The Year,2018,General,this is America,Childish Gambino
1,Song Of The Year,2018,General,this is America,Childish Gambino
2,Best Pop Solo Performance,2018,Pop,Joanne (where Do you Think You're Goin'?),Lady Gaga
3,Best Pop Duo/Group Performance,2018,Pop,Shallow,Lady Gaga & Bradley Cooper
4,Best Dance Recording,2018,Dance/Electronic Music,Electricity,Silk City & Dua Lipa Featuring Diplo & Mark Ro...
...,...,...,...,...,...
423,Best Rap Solo Performance,1999,Rap,My Name is,Eminem
424,Best Rap Performance By A Duo Or Group,1999,Rap,you Got Me,The roots featuring Erykah Badu
425,Best Female Country Vocal Performance,1999,Country,Man! i Feel Like a Woman!,Shania Twain
426,Best Male Country Vocal Performance,1999,Country,Choices,George Jones


In [27]:
#get rid of all na columns
Adf=Adf.dropna()
Bdf=Bdf.dropna()
Gdf=Gdf.dropna()
#changed all strings to lower case
Gdf = Gdf.astype(str).apply(lambda x: x.str.lower())
Adf = Adf.astype(str).apply(lambda x: x.str.lower())
Bdf = Bdf.astype(str).apply(lambda x: x.str.lower())

In [28]:
#added all rewards in one cell for the same artist
Gdf1=Gdf[['GrammyAward','Artist']]
Gdf1=Gdf1.groupby('Artist')['GrammyAward'].apply(', '.join).reset_index()

In [29]:
#getting rid of multiple artists in one cell
Bdf=Bdf[~Bdf['Artists'].str.contains(",")]
Bdf

,Artists,Name,Weekly_rank,Peak_position,Date,Genre
2,billie eilish,bad guy,3,2.0,"march 29, 2019","hip-hop,dark pop,house,trap,memes,alternative ..."
3,khalid,talk,4,3.0,"february 7, 2019","synth-pop,pop"
5,jonas brothers,sucker,6,1.0,"march 1, 2019","alternative pop,boy band,teen pop,pop-rock,pop"
6,dababy,suge,7,7.0,"march 1, 2019","trap,east coast,rap"
7,drake,money in the grave,8,7.0,"june 15, 2019","hip-hop,rap,basketball,nba,canada"
...,...,...,...,...,...,...
24994,usher,good kisser,74,66.0,"may 5, 2014","r&;b,pop"
24995,schoolboy q,studio,75,75.0,"february 25, 2014","west coast,rap"
24996,dustin lynch,where it's at,76,76.0,"march 31, 2014","rock,country"
24998,lorde,tennis court,78,71.0,"june 7, 2013","alternative pop,alternative,downtempo,new zeal..."


In [30]:
rds_connection_string = f"postgres:{password}@localhost:5432/musician_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [31]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) fe_sendauth: no password supplied

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
Adf.to_sql(name='Artists', con=engine, if_exists='append', index=False)
Bdf.to_sql(name='Billboard', con=engine, if_exists='append', index=False)
Gdf.to_sql(name='Grammy', con=engine, if_exists='append', index=False)